## 확률값이 저장된 csv 파일 만들기 (csv -> parquet)
확률값으로 앙상블하기 위해서 model을 불러와서 inference 과정을 다시 진행해야합니다.  
모든 픽셀값을 저장하기엔 메모리 부족으로 threshold를 0.3으로 설정하여 해당 값을 초과한 확률값만 저장합니다.
* 주의 : 모델을 학습시킬때의 동일한 Resize 크기로 inference를 진행해야합니다.
* 파일을 읽는 시간 단축을 위해 format을 csv에서 parquet으로 저장하였습니다.

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import albumentations as A
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm.auto import tqdm
import cv2

if not os.path.exists('./ensemble_parquet'):
    os.makedirs('./ensemble_parquet')

# ============ TODO ===============

model_file = '/opt/ml/input/code/checkpoints/[final]CJE/best_model.pt'  # 모델 weight 경로 설정
save_parquet = './ensemble_parquet/'+ '[final]CJE.parquet' # 저장할 파일명 설정 (format : '.parquet')
IMAGE_ROOT = "/opt/ml/input/data/test/DCM"  # test dataset 경로
tf = A.Compose([     # Transform 설정 - 학습시 설정했던 Transform과 동일해야함
        A.Resize(1024, 1024),
])
# =================================

In [ ]:
pngs = {
    os.path.relpath(os.path.join(root, fname), start=IMAGE_ROOT)
    for root, _dirs, files in os.walk(IMAGE_ROOT)
    for fname in files
    if os.path.splitext(fname)[1].lower() == ".png"
}
print(len(pngs))  # 총 300개

In [ ]:
CLASSES = [
    'finger-1', 'finger-2', 'finger-3', 'finger-4', 'finger-5',
    'finger-6', 'finger-7', 'finger-8', 'finger-9', 'finger-10',
    'finger-11', 'finger-12', 'finger-13', 'finger-14', 'finger-15',
    'finger-16', 'finger-17', 'finger-18', 'finger-19', 'Trapezium',
    'Trapezoid', 'Capitate', 'Hamate', 'Scaphoid', 'Lunate',
    'Triquetrum', 'Pisiform', 'Radius', 'Ulna',
]
CLASS2IND = {v: i for i, v in enumerate(CLASSES)}
IND2CLASS = {v: k for k, v in CLASS2IND.items()}

def encode_mask_to_rle(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    prob = []
    runs = np.where((pixels[1:]>0) != (pixels[:-1]>0))[0] + 1   # 0이 아닌 연속적인 값의 시작 좌표 저장
    runs[1::2] -= runs[::2]
    for x, y in zip(runs[::2], runs[1::2]):
        for i in range(y):
            prob.append(round(pixels[x+i], 2))  # 확률값을 소수점 둘째자리까지 반올림한 값 저장
    return ' '.join(str(x) for x in runs), ' '.join(str(x) for x in prob)

class XRayInferenceDataset(Dataset):
    def __init__(self, transforms=None):
        _filenames = pngs
        _filenames = np.array(sorted(_filenames))

        self.filenames = _filenames
        self.transforms = transforms

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, item):
        image_name = self.filenames[item]
        image_path = os.path.join(IMAGE_ROOT, image_name)

        image = cv2.imread(image_path)
        image = image / 255.
        # image = image.astype("float32")

        if self.transforms is not None:
            inputs = {"image": image}
            result = self.transforms(**inputs)
            image = result["image"]

        # to tenser will be done later
        image = image.transpose(2, 0, 1)    # make channel first

        image = torch.from_numpy(image).float()

        return image, image_name

def test(model, data_loader, thr=0.5):
    model = model.cuda()
    model.eval()

    rles = []
    probs = []
    filename_and_class = []
    with torch.no_grad():
        n_class = len(CLASSES)

        for step, (images, image_names) in tqdm(enumerate(data_loader), total=len(data_loader)):
            images = images.cuda()
            outputs = model(images)['out']

            # restore original size
            outputs = F.interpolate(outputs, size=(2048, 2048), mode="bilinear")
            outputs = torch.sigmoid(outputs)
            outputs = outputs.detach().cpu().numpy()
            outputs = np.where(outputs > 0.3, outputs, 0)   # threshold=0.3 초과면 확률값을, 아니라면 0으로 변환

            for output, image_name in zip(outputs, image_names):
                for c, segm in enumerate(output):
                    rle, prob = encode_mask_to_rle(segm)   # rle 갯수에 맞는 확률값도 불러옴
                    rles.append(rle)
                    probs.append(prob)
                    filename_and_class.append(f"{IND2CLASS[c]}_{image_name}")

    return rles, probs, filename_and_class

In [ ]:
test_dataset = XRayInferenceDataset(transforms=tf)
test_loader = DataLoader(
    dataset=test_dataset, 
    batch_size=2,
    shuffle=False,
    num_workers=2,
    drop_last=False
)

In [ ]:
# model = torch.load(model_file)['model']
from torchvision import models
model = models.segmentation.fcn_resnet50(pretrained=True)
model.classifier[4] = nn.Conv2d(512, 29, kernel_size=1)
model.load_state_dict(torch.load(model_file)["model"])

In [ ]:
rles, probs, filename_and_class = test(model, test_loader)

In [ ]:
classes, filename = zip(*[x.split("_") for x in filename_and_class])
image_name = [os.path.basename(f) for f in filename]
df = pd.DataFrame({
    "image_name": image_name,
    "class": classes,
    "rle": rles,
    "prob": probs,    # 확률값 저장 추가
})

In [ ]:
df.head()

In [ ]:
# df.to_csv("output.csv", index=False)
df.to_parquet(save_parquet, compression='gzip')   # 시간 단축을 위한 parquet(파케이) 포맷으로 저장 (읽을때 약 5배 시간단축)